# Conversion of Height data from LIDAR LAS file

Romain Beucher, July 2024

In [1]:
import laspy

## Explore data from the lasfile

In [2]:
las_file_path = "Aramac_ell/SW_316000_7461000_1k_class_Ellipsoid.las"

In [3]:
input_lasfile = laspy.open(las_file_path)
points = input_lasfile.read()
x_coords = points.x
y_coords = points.y
z_coords = points.z

In [4]:
points.x.min(), points.x.max()

(316000.0, 316999.999)

In [5]:
points.X.min(), points.X.max()

(316000000, 316999999)

In [6]:
points.y.min(), points.y.max()

(7461000.0, 7461999.999)

In [7]:
points.Y.min(), points.Y.max()

(461000000, 461999999)

### Number of points

In [8]:
len(points.x)

11951102

Here we can retrieve the authority

In [9]:
projcs = input_lasfile.header.vlrs[1].parse_crs()
print(projcs)

PROJCS["GDA2020 / GDA2020 / MGA 55S",GEOGCS["GDA2020",DATUM["Geocentric_Datum_of_Australia_2020",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","9844"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","7844"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",147],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","7855"]]


In [10]:
#!pip install pyproj
projcs = input_lasfile.header.vlrs[1].parse_crs().to_authority()
epsg_code = int(projcs[1])

print(epsg_code)

7855


In [11]:
list(input_lasfile.header.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time']

In [12]:
list(input_lasfile.header.point_format.extra_dimension_names)

[]

## Corrdinate conversion 

We need to convert the coordinates to lat/lon in order to use the geodepy functions

In [13]:
from osgeo import ogr, osr

def convert_coordinates(coordinates, input_epsg=7855, output_epsg=4326):
    # Create coordinate transformation
    input_spatial_ref = osr.SpatialReference()
    input_spatial_ref.ImportFromEPSG(input_epsg)

    output_spatial_ref = osr.SpatialReference()
    output_spatial_ref.ImportFromEPSG(output_epsg)

    coord_transform = osr.CoordinateTransformation(input_spatial_ref, output_spatial_ref)
    
    # Transform point
    return coord_transform.TransformPoints(coordinates)

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame()

In [16]:
df["X"] = points.X * points.x.scale + points.x.offset
df["Y"] = points.Y * points.y.scale + points.y.offset
df["GPS_Height"] = points.Z * points.z.scale + points.z.offset
df[["Lat", "Lon", "unknown"]] = convert_coordinates(list(zip(df.X, df.Y)), input_epsg=epsg_code)
df = df.drop(columns=["unknown"])

/home/romain/.conda/envs/geodepy_env/lib/python3.8/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [17]:
df.head()

,X,Y,GPS_Height,Lat,Lon
0,316000.000,7461207.518,257.376,-22.947145,145.205458
1,316000.000,7461178.164,257.183,-22.947410,145.205455
2,316000.000,7461057.969,256.524,-22.948496,145.205440
3,316415.653,7461000.000,256.707,-22.949065,145.209486
4,316294.803,7461000.000,256.575,-22.949052,145.208308


## Convert Height

The following code is probably not suitable for processing millions of points. We could use a dask cluster or work on improving the Geodepy function. For now, I am just leaving it as a proof of concept and only process the first 5 points.

In [18]:
npoints = 5
df = df.head(npoints) # Select only the first 5 points

In [19]:
from geodepy.height import GPS_to_AVWS, GPS_to_AHD, GPS_to_AUSGeoid09, GPS_to_AUSGeoid98

results = df.apply(lambda row: GPS_to_AVWS(row["Lat"], row["Lon"], row["GPS_Height"]), axis=1)
df[['AVWS_Height', 'AVWS_Height_stderr']] = pd.DataFrame(results.to_list(), index = df.index).applymap(lambda x: x[0])

df.head()

,X,Y,GPS_Height,Lat,Lon,AVWS_Height,AVWS_Height_stderr
0,316000.000,7461207.518,257.376,-22.947145,145.205458,214.020373,0.055
1,316000.000,7461178.164,257.183,-22.947410,145.205455,213.828771,0.055
2,316000.000,7461057.969,256.524,-22.948496,145.205440,213.175485,0.055
3,316415.653,7461000.000,256.707,-22.949065,145.209486,213.348148,0.055
4,316294.803,7461000.000,256.575,-22.949052,145.208308,213.219968,0.055


In [20]:
points.y.offset

7000000.0

## Write output to new LAS file

In [21]:
import numpy as np
from copy import copy

# Prepare the header for the output LAS file
header = copy(input_lasfile.header)
header.point_count = 0

# Create las file
new_file = laspy.LasData(header)
# LAS file only accept coordinates as integer, hence scaling and offset parameters. Here I use the ones from the original file.
new_file.X = points.X[:npoints]
new_file.Y = points.Y[:npoints]
new_file.Z = ((df["AVWS_Height"] - points.z.offset) / points.z.scale ).astype(int)
# Write to disk
new_file.write('extracted_points.las')

In [22]:
((df["Y"] - points.y.offset) / points.y.scale).astype(int)

0    461207518
1    461178163
2    461057968
3    461000000
4    461000000
Name: Y, dtype: int64

In [23]:
points.Y

array([461207518, 461178164, 461057969, ..., 461999776, 461999917,
       461999933], dtype=int32)

In [24]:
points.y.offset, new_file.y.offset

(7000000.0, 7000000.0)

In [25]:
list(new_file.header.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time']

## Open new LAS file for testing

In [26]:
las_file_path = 'extracted_points.las'

In [27]:
lasfile = laspy.open(las_file_path)
points = lasfile.read()
x_coords = points.x
y_coords = points.y
z_coords = points.z

In [28]:
points.Y.min(), points.Y.max()

(461000000, 461207518)

In [29]:
len(points.x) # only contains 5 points as expected

5

In [30]:
list(lasfile.header.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time']

In [31]:
one_point = points[0]

In [32]:
list(one_point.header.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time']

In [33]:
points.y.min(), points.y.max()

(7461000.0, 7461207.518)

In [34]:
points.y.offset

7000000.0